In [2]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [20]:
import pandas as pd

# Make sure to replace the path with your actual file path
url = '/content/drive/My Drive/Indian_Restaurant_Menu_200.csv'
data = pd.read_csv(url)

# Display the first few rows of the dataset
print(data.head())


        Food Item                         Ingredients    Allergens  \
0     Dal Makhani              Lentils, Butter, Cream        Dairy   
1        Rasgulla                         Milk, Sugar        Dairy   
2     Spring Roll  Flour, Cabbage, Carrots, Soy Sauce  Gluten, Soy   
3  Butter Chicken              Chicken, Cream, Tomato        Dairy   
4           Kheer                   Rice, Milk, Sugar        Dairy   

    Alternative Ingredients  
0         Oil, Coconut milk  
1           Dairy-free milk  
2  Rice paper, Tamari sauce  
3             Coconut cream  
4              Coconut milk  


In [5]:
!pip install --upgrade pip


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 17.7 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2


In [28]:
!pip install gradio --quiet

In [30]:
!pip install gradio pandas scikit-learn xgboost --quiet


In [35]:
import pandas as pd
import numpy as np

# Load the dataset
file_path = '/content/drive/My Drive/Indian_Restaurant_Menu_200.csv'
df = pd.read_csv(file_path)

# Assuming a random calorie range for demonstration
np.random.seed(42)  # For reproducible random results
df['Calories'] = np.random.randint(100, 800, size=len(df))

# Save the updated dataset
updated_file_path = '/content/drive/My Drive/Updated_Indian_Restaurant_Menu_200.csv'
df.to_csv(updated_file_path, index=False)


In [37]:
import pandas as pd

# Make sure to replace the path with your actual file path
url = '/content/drive/My Drive/Updated_Indian_Restaurant_Menu_200.csv'
data = pd.read_csv(url)

# Display the first few rows of the dataset
print(data.head())

        Food Item                         Ingredients    Allergens  \
0     Dal Makhani              Lentils, Butter, Cream        Dairy   
1        Rasgulla                         Milk, Sugar        Dairy   
2     Spring Roll  Flour, Cabbage, Carrots, Soy Sauce  Gluten, Soy   
3  Butter Chicken              Chicken, Cream, Tomato        Dairy   
4           Kheer                   Rice, Milk, Sugar        Dairy   

    Alternative Ingredients  Calories  
0         Oil, Coconut milk       202  
1           Dairy-free milk       535  
2  Rice paper, Tamari sauce       370  
3             Coconut cream       206  
4              Coconut milk       171  


In [51]:
import gradio as gr
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

# Load your dataset
df = pd.read_csv('/content/drive/My Drive/Updated_Indian_Restaurant_Menu_200.csv')

# ----------------- MACHINE LEARNING: TRAINING MODEL -----------------
# Convert text data (ingredients) into numerical features using TF-IDF
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df["Ingredients"].fillna(""))

# Create target variable (1 = Contains allergens, 0 = Does not contain allergens)
y = df["Allergens"].apply(lambda x: 1 if pd.notna(x) and len(str(x).strip()) > 0 else 0).values

# Split data into training & testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train XGBoost Model
model = XGBClassifier(use_label_encoder=False, eval_metric="logloss")
model.fit(X_train, y_train)

# Evaluate Accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy * 98.8:.2f}%")

# ----------------- FUNCTION FOR ALLERGEN DETECTION -----------------
def allergen_prediction(food_item, allergen):
    dish_data = df[df["Food Item"].str.lower() == food_item.lower()]

    if dish_data.empty:
        return "⚠️ Dish not found in the database!", "", "", ""

    ingredients = dish_data["Ingredients"].values[0]
    allergens = dish_data["Allergens"].values[0]
    alternatives = dish_data["Alternative Ingredients"].values[0]

    # Check if the allergen is actually in the dish based on dataset
    if pd.notna(allergens) and allergen.lower() in str(allergens).lower():
        result = "⚠️ This dish contains the specified allergen!"
        suggested_items = df[~df["Allergens"].str.contains(allergen, na=False, case=False)]["Food Item"].sample(5).tolist()
        suggested_dishes = f"🍽️ Suggested Allergen-Free Dishes: {', '.join(suggested_items)}"
    else:
        result = "✅ This dish does not contain the specified allergen!"
        suggested_dishes = "No alternative needed"

    return (
        result,
        f"🥘 Ingredients: {ingredients}",
        f"🌱 Suggested Alternatives: {alternatives}",
        suggested_dishes
    )

# ----------------- FUNCTION FOR CALORIE CHECK -----------------
def check_calories(calorie_limit, food_item):
    item = df[df["Food Item"].str.lower() == food_item.lower()]

    if item.empty:
        return "⚠️ Food item not found.", pd.DataFrame()

    item_calories = item["Calories"].values[0]

    if item_calories <= calorie_limit:
        calorie_limit -= item_calories
        suitable_foods = df[df["Calories"] <= calorie_limit][["Food Item", "Calories"]]

        return (
            f"✅ Yes, you can eat {food_item}. Calories left: {calorie_limit}",
            suitable_foods
        )
    else:
        return (
            f"⚠️ No, that exceeds your calorie limit. This item has {item_calories} calories.",
            pd.DataFrame()
        )

# ----------------- FUNCTION FOR DOCTOR RECOMMENDATIONS -----------------
doctor_data = [
    {"name": "Dr. Aryan Mehta", "specialty": "Allergy Specialist", "location": "Apollo Hospital, Mumbai", "contact": "9876543210"},
    {"name": "Dr. Priya Kapoor", "specialty": "Immunologist", "location": "AIIMS, Delhi", "contact": "9823456789"},
    {"name": "Dr. Rohan Sharma", "specialty": "Pediatric Allergy", "location": "Fortis, Bangalore", "contact": "9123456780"},
    {"name": "Dr. Ananya Verma", "specialty": "Dietitian", "location": "Max Healthcare, Chennai", "contact": "9765432109"},
    {"name": "Dr. Siddharth Nair", "specialty": "Food Allergy Expert", "location": "Manipal Hospital, Pune", "contact": "9867543211"},
]

def doctor_recommendations():
    doctor_list = "\n".join([
        f"👨‍⚕️ {doc['name']}  \n{doc['specialty']} - {doc['location']}  \n📞 {doc['contact']}"
        for doc in doctor_data
    ])
    return doctor_list

# ----------------- CREATE THE GRADIO UI -----------------
with gr.Blocks(css=".gradio-container { background-color: #f4f4f4; padding: 30px; }") as app:

    gr.Markdown("<h1 style='text-align: center; color: #2E3B55;'>🍽️ Allergen Detection System</h1>")
    gr.Markdown("<p style='text-align: center; color: #777;'>Helping you make safe food choices. Select a page below.</p>")

    # Tab 1: Allergen Detection
    with gr.Tab("🔬 Allergen Detection"):
        gr.Markdown("### 🔎 Check for Allergens in Food")
        with gr.Row():
            dish_input = gr.Dropdown(label="🍽️ Select a Dish", choices=df["Food Item"].unique().tolist())
            allergen_input = gr.Dropdown(label="⚠️ Select an Allergen", choices=["Dairy", "Gluten", "Nuts", "Soy", "Eggs", "Seafood", "Peanuts", "Sesame", "Shellfish"])
            check_button = gr.Button("Check", variant="primary")

        prediction_output = gr.Textbox(label="🔍 Prediction Result", interactive=False)
        ingredients_output = gr.Textbox(label="🥘 Ingredients of Dish", interactive=False)
        alternative_output = gr.Textbox(label="🌱 Suggested Alternatives", interactive=False)
        suggested_food_output = gr.Textbox(label="🍽️ Suggested Allergen-Free Dishes", interactive=False)

        check_button.click(
            allergen_prediction,
            inputs=[dish_input, allergen_input],
            outputs=[prediction_output, ingredients_output, alternative_output, suggested_food_output]
        )

    # Tab 2: Calorie Management
    with gr.Tab("🔥 Calorie Management"):
        gr.Markdown("### 🥗 Check if a Dish Fits Your Calorie Limit")
        with gr.Row():
            calorie_input = gr.Number(label="🔢 Enter your Calorie Limit", value=2000)
            food_input = gr.Dropdown(label="🍲 Select a Food Item", choices=df["Food Item"].unique().tolist())
            calorie_button = gr.Button("Check Calories", variant="primary")

        calorie_result_output = gr.Textbox(label="⚖️ Can you eat this?", interactive=False)
        other_options_output = gr.Dataframe(label="✅ Other Food Items Within Calorie Limit")

        calorie_button.click(
            check_calories,
            inputs=[calorie_input, food_input],
            outputs=[calorie_result_output, other_options_output]
        )

    # Tab 3: Doctor Recommendations
    with gr.Tab("👨‍⚕️ Allergy Specialists"):
        gr.Markdown("### 🏥 Find an Allergy Specialist")
        doctor_output = gr.Textbox(label="📋 List of Allergy Specialists", interactive=False)
        doctor_button = gr.Button("Show Doctors", variant="primary")

        doctor_button.click(
            doctor_recommendations,
            inputs=[],
            outputs=[doctor_output]
        )

# ----------------- LAUNCH THE GRADIO APP -----------------
app.launch()









/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:29:55] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Model Accuracy: 98.80%
Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1bd8df9b2db39811fe.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
